In [29]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [30]:
dataset_path = './realwaste/realwaste-main/RealWaste'

In [31]:
def get_dataset_partitions(ds, train_split=0.6, val_split=0.2, test_split=0.2, shuffle=True, shuffle_size = 10000):

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [32]:
# Load the dataset from a directory (adjust path)
dataset_path = './realwaste/realwaste-main/RealWaste'

# Use TensorFlow's image_dataset_from_directory to load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(524, 524),  # Resize all images to 128x128
    batch_size=32,          # Batch size for loading
    label_mode='int'        # Use integer labels
)

# Partition the dataset into training, validation, and test sets
train_ds, val_ds, test_ds = get_dataset_partitions(dataset, train_split=0.6, val_split=0.2, test_split=0.2)


Found 4752 files belonging to 9 classes.


In [33]:
AUTOTUNE = tf.data.AUTOTUNE  # Auto-tune for performance optimization

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize image to [0, 1]
    image = tf.image.resize(image, [128, 128])  # Resize image to 128x128
    return image, label

# Apply preprocessing
train_ds = train_ds.map(preprocess).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)


In [34]:
# Load the pre-trained ResNet50 model with ImageNet weights (excluding the top classification layers)
resnet_base = tf.keras.applications.ResNet50(
    include_top=False,          # Exclude top layers (we'll add our own)
    weights='imagenet',         # Load pre-trained weights from ImageNet
    input_shape=(128, 128, 3)   # Input size (128x128 images with 3 channels)
)

# Freeze the layers of ResNet50 to prevent updating during initial training
resnet_base.trainable = False

from tensorflow.keras import layers, models

# Build the model using tf.keras
model = models.Sequential([
    resnet_base,  # Add pre-trained ResNet50 as the base model
    layers.GlobalAveragePooling2D(),  # Pool feature maps into a 1D vector
    layers.Dense(128, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),  # Dropout layer
    layers.Dense(len(dataset.class_names), activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy for integer labels
    metrics=['accuracy']
)


In [ ]:
class CustomTrainingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_loss = logs.get('loss', 0)
        train_accuracy = logs.get('accuracy', 0) * 100  # Convert to percentage
        val_loss = logs.get('val_loss', 0)
        val_accuracy = logs.get('val_accuracy', 0) * 100  # Convert to percentage
        
        print(f"Epoch {epoch + 1}/{self.params['epochs']}, "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% "
              f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Assuming `model`, `train_ds`, and `val_ds` are already defined
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,  # number of epochs
    verbose=0,  
    callbacks=[CustomTrainingCallback()] 
)


2024-12-12 13:52:11.176173: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 105437312 bytes after encountering the first element of size 105437312 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()
